In [1]:
import numpy as np
import pandas as pd
import copy
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv("csv\data_test.csv", sep=';')

In [3]:
df.head(10)

,id события,ФИО участника события 1,ФИО участника события 2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна
5,595,Русских Кира Константиновна,Федутинов Артем Артурович
6,637,Федонкина Кристина Маратовна,Маргиев Григорий Иванович
7,741,Чернолусская Екатерина Марселевна,Бухановская Лидия Радиковна
8,996,Оранский Владимир Артурович,Скотников Виктор Ильич
9,1210,Ардатов Андрей Анатольевич,Ханыгин Дмитрий Юрьевич


<i> Для удобства переименуем колонки </i>

In [4]:
df = df.rename(columns={'id события':'id', 'ФИО участника события 1':'name_1', 'ФИО участника события 2':'name_2'})

In [5]:
df.head(10)

,id,name_1,name_2
0,189,Галчевская Карина Владимировна,Белоновская Анастасия Семеновна
1,206,Офицеров Олег Романович,Сапожник Борис Валерьевич
2,445,Жандарова Лариса Германовна,Чемодуров Дамир Русланович
3,503,Масимова Яна Дамировна,Мингажетдинов Рамиль Семенович
4,571,Мухтарова Алена Яковлевна,Щербатенко Ольга Робертовна
5,595,Русских Кира Константиновна,Федутинов Артем Артурович
6,637,Федонкина Кристина Маратовна,Маргиев Григорий Иванович
7,741,Чернолусская Екатерина Марселевна,Бухановская Лидия Радиковна
8,996,Оранский Владимир Артурович,Скотников Виктор Ильич
9,1210,Ардатов Андрей Анатольевич,Ханыгин Дмитрий Юрьевич


#### Поиск дубликатов

In [6]:
df.id.nunique() == df.shape[0]

False

In [7]:
duplicated_id = df.loc[df.duplicated(['id'])]

In [8]:
dupl_df_id = df.loc[df.id.isin(duplicated_id.id)]

In [9]:
dupl_df_id.name_1.nunique() == dupl_df_id.name_2.nunique() == dupl_df_id.shape[0]

True

In [10]:
len(set(dupl_df_id.name_1.unique()).intersection(set(dupl_df_id.name_2.unique())))

0

In [11]:
dupl_df_id.head(10)

,id,name_1,name_2
366,70049,Яшина Полина Евгеньевна,Герасимовская Ксения Дамировна
367,70049,Федова Анжелика Вадимовна,Вальдовский Альберт Ефимович
486,92995,Журик Альберт Евгеньевич,Бадьянова Римма Максимовна
487,92995,Кучеренко Ирина Ильинична,Болтик Григорий Максимович
597,117280,Волынский Кирилл Федорович,Гайсумов Виктор Тимурович
598,117280,Уточкин Евгений Анатольевич,Каганович Лилия Петровна
899,177407,Зелинский Геннадий Артурович,Зазорин Вадим Аркадьевич
900,177407,Бужанинов Руслан Артурович,Сороковой Герман Маратович
1798,358194,Ноева Галина Степановна,Серпухова Алла Ярославовна
1799,358194,Долгих Лилия Вадимовна,Брусенцова Дарья Михаиловна


In [12]:
df_id_duplicates = df.pivot_table(index='id', aggfunc='size')

In [13]:
df_id_duplicates.unique()

array([1, 2], dtype=int64)

<i>Существует от 2-х до 4-х людей привязанных к одному и тому же id события.<br>
Повторений фамилий для повторяющихся id не обнаружено.</i>


-----------

#### Примем, что люди с идентичными ФИО одни и те же люди

In [14]:
serie_names = pd.concat([df.name_1, df.name_2], axis=0, names=['idx','event']).reset_index()

In [15]:
serie_names_dupl = serie_names.pivot_table(index=[0], aggfunc='size')

In [16]:
serie_names_dupl.unique()

array([ 1,  2, 50, 14,  3,  5,  6,  4], dtype=int64)

In [17]:
serie_names_dupl = serie_names_dupl.sort_values(ascending=False)

In [18]:
serie_names_dupl.head(10)

0
Ахромеева Алина Ивановна            50
Башнина Антонина Глебовна           14
Медведева Дарья Алексеевна           6
Диомидов Игорь Ильдарович            5
Зимнухова Карина Даниловна           5
Шолохов Игорь Робертович             4
Двигубская Валентина Геннадьевна     3
Пафомова Кира Вадимовна              3
Анихнова Тамара Руслановна           2
Каехтин Ильдар Эдуардович            2
dtype: int64

<i>Топ 10 самых активных участников</i>

In [19]:
s1 = set(df.name_1.unique())
s2 = set(df.name_2.unique())
eventsIntersection = s1.intersection(s2)

In [20]:
len(eventsIntersection)

25

<i> 25 людей повторяются как в первом, так и во втором столбце events

-----

<i>Резюме по информации, которую удалось достать из датасета
<ul>
    <li> Существует от 2-х до 4-х людей привязанных к одному и тому же id события.
    <li> Для всех повторяющихся id событий не было обнаружено дубликатов ФИО
    <li> Самый активный участник - Ахромеева Алина Ивановна
</ul>

#### Загрузка информации в графовую БД

In [160]:
unique_name_nodes = serie_names[0].unique()
unique_ids = df.id.unique()

In [161]:
from neo4j import GraphDatabase


def executeTransactions(transList : list):
    database_connection = GraphDatabase.driver(uri="bolt://localhost:7687", auth=("neo4j", "admin"))
    session = database_connection.session()
    for block in transList:
        for i in block:
            session.run(i)

In [166]:
def create_id_nodes_req_list(id_nodes_list : list)->list:
    req_list = list()
    for i in id_nodes_list:
        req_list.append(f'CREATE(:event_id{{number:{i}}})')
        
    return req_list


def create_people_nodes_req_list(event_nodes_list : list)->list:
    req_list = list()
    for i in event_nodes_list:
        req_list.append(f'CREATE(:participant{{name:"{i}"}})')
        
    return req_list


def set_relations_between_nodes(listedDF : list)->list:
    req_list = list()
    
    for row in listedDF:
        for i in range(1,3):
            req_list.append(f'MATCH(e:event_id),(p:participant)\
                            WHERE e.number = {row[0]} AND p.name = "{row[i]}"\
                            CREATE (p)-[:takes_part_in]->(e)')
    
    
    return req_list

In [167]:
listedDF = df.values.tolist()

In [168]:
req_list = list()
req_list.append(create_id_nodes_req_list(unique_ids))
req_list.append(create_people_nodes_req_list(unique_name_nodes))
req_list.append(set_relations_between_nodes(listedDF))

In [169]:
executeTransactions(req_list)

----

<i> Примеры запросов к графовой БД</i>

![title](img\11.jpg)

![title](img\22.jpg)

<i>Работа сервиса REST api на Flask </i>

![title](img\33.jpg)

![title](img\44.jpg)